In [1]:
from numpy import pi, linspace, diff, mean, min, max, arctan2, dot
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
planet_name = "moon"
ref_name = "earth"
ts = load.timescale()
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(1900, 1, 1, 0, 0, 0)
period_days = 365*8
eph_s = load(eph_s_name)
earth = eph_s["earth"]
sun = eph_s["sun"]
planet = eph_s[planet_name]
ref = eph_s[ref_name]

def get_fill_colors(times):
    return [f"#{255:02x}{int(255 * (times.tt[-1] - t) / (times.tt[-1] - times.tt[0])):02x}{255:02x}" for t in times.tt]

## Distance from Earth

In [3]:
times_plot = ts.tt_jd(linspace(t_start.tt, t_start.tt + period_days, 10000))
lat, lon, dist = (planet.at(times_plot) - ref.at(times_plot)).frame_latlon(ecliptic_J2000_frame)
pos_vec, v_vec = (planet.at(times_plot) - ref.at(times_plot)).frame_xyz_and_velocity(ecliptic_J2000_frame)

# mean(dist.au), min(dist.au), max(dist.au)
mean(dist.km), min(dist.km), max(dist.km)

(np.float64(385012.04354818194),
 np.float64(356705.897997824),
 np.float64(406707.1207792721))

In [4]:
fig = figure(title=planet_name.capitalize() + " distance", x_axis_label="days", y_axis_label="distance (km)", width=1100)
fig.scatter(times_plot.tt-t_start.tt, dist.km, line_color="red", fill_color = get_fill_colors(times_plot))
fig.line(times_plot.tt-t_start.tt, dist.km, line_color="red")
show(fig)

In [5]:
fig = figure(title=planet_name.capitalize() + " dist vel", x_axis_label="days", y_axis_label="~distance vel", width=1100)
v_x, v_y, v_z = v_vec.km_per_s
x, y, z = pos_vec.km
fig.scatter(times_plot.tt-t_start.tt, v_x*x + v_y*y + v_z*z, line_color="red", fill_color = get_fill_colors(times_plot))
fig.line(times_plot.tt-t_start.tt, v_x*x + v_y*y + v_z*z, line_color="red")
show(fig)

In [6]:
def dist_vel_zero(target, ref):
    def zero(t):
        pos_vec, v_vec = (planet.at(t) - ref.at(t)).frame_xyz_and_velocity(ecliptic_J2000_frame)
        v_x, v_y, v_z = v_vec.km_per_s
        x, y, z = pos_vec.km
        return v_x*x + v_y*y + v_z*z > 0
    zero.step_days = 10.0
    return zero

In [7]:
vel_zero = find_discrete(t_start, t_start + period_days, dist_vel_zero(planet, ref))
t_dist_max = vel_zero[0][vel_zero[1] == 0]
lat, lon, dist = (planet.at(t_dist_max) - ref.at(t_dist_max)).frame_latlon(ecliptic_J2000_frame)
pos_max_vel, v_max_vel = (planet.at(t_dist_max) - ref.at(t_dist_max)).frame_xyz_and_velocity(ecliptic_J2000_frame)
# mean(diff(vel_zero_node)), min(diff(vel_zero_node)), max(diff(vel_zero_node)), len(diff(vel_zero_node))

In [8]:
fig = figure(title=planet_name.capitalize()+" max dist location", x_axis_label="x(km)", y_axis_label="y(km)", width=1000, height=1000)
fig.scatter(pos_max_vel.km[0], pos_max_vel.km[1], fill_color=get_fill_colors(t_dist_max), line_color="red")
fig.line(pos_max_vel.km[0], pos_max_vel.km[1], line_color="red")
show(fig)

In [9]:
fig = figure(title=planet_name.capitalize()+" max distance", x_axis_label="days", y_axis_label="km", width=1100)
fig.scatter(t_dist_max.tt - t_start.tt, dist.km, fill_color=get_fill_colors(t_dist_max), line_color="red")
fig.line(t_dist_max.tt - t_start.tt, dist.km, line_color="red")
show(fig)

In [10]:
fig = figure(title=planet_name.capitalize()+" max distance long, lat", x_axis_label="days", y_axis_label="", width=1100)
fig.scatter(t_dist_max.tt - t_start.tt, lat.degrees, fill_color=get_fill_colors(t_dist_max), line_color="red", legend_label = "latitude(deg)")
fig.line(t_dist_max.tt - t_start.tt, lat.degrees, line_color="red", legend_label = "latitude(deg)")
fig.scatter(t_dist_max.tt - t_start.tt, lon.hours, fill_color=get_fill_colors(t_dist_max), line_color="blue", legend_label = "longitude(hours)")
fig.line(t_dist_max.tt - t_start.tt, lon.hours, line_color="blue", legend_label = "longitude(hours)")
show(fig)